## The program takes a markdown table output (from CHATGPT in a separate browser) then converts to a usable table.
#### Example of a markdown text input below (toggle on configure to see it).

| Table Name (UI) | Technical Name | Description |
| --- | --- | --- |
| Business Unit | cmn_cost_center | Contains records for each business unit within your organization. Business units can be used for categorizing costs, work, and other elements at a more granular level than a company. |
| Department | cmn_department | Contains records for each department within your organization. Each user can be assigned to a department, which can be used for routing decisions, reporting, and other functions. |
| Location | cmn_location | Contains records for physical locations where your company operates. This can include offices, data centers, or other locations. Each user and asset can be associated with a location. |
| Company | core_company | Contains records for each company that your organization interacts with. This can include your own company, vendors, partners, and customers. |
| User | sys_user | Contains a record for each individual who has (or had) access to your ServiceNow instance. It includes information like their name, email, roles, etc. |
| Group | sys_user_group | Contains records for each group of users within your ServiceNow instance. Groups can be used for many purposes, like assignment, approval rules, notifications, and access control. |
| Group Members | sys_user_grmember | Contains records of the relationships between users and groups. Each record represents a user being a member of a group. |
| Role | sys_user_role | Contains records for each role within your ServiceNow instance. Roles are used to define what actions a user can perform and what data they can access. |

In [1]:
import pandas as pd
from io import StringIO
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
markdown_table = """
| Term | Definition | Notes |
| --- | --- | --- |
| Portfolio | Collection of services, products, projects, or applications. | Used to manage like items together for a business. Portfolios may be grouped by objective, capabilities, organization, like projects or services. |
| Business capability | High-level capability that an organization requires to execute its business model or fulfill its mission. | Typically described in the context of performing one or more specific tasks to achieve business outcomes. For example, demand management or financial planning. |
| Service | Means of delivering value to customers by facilitating outcomes customers want to achieve without the ownership of specific costs and risks. | ServiceNow provides three base service types: Business, Technical, Application. Customers can extend the base types to align with the service types in their organization. |
| Business service | A service type that is published to business users and it typically underpins one or more business capabilities. | Typically orderable by business users. Business users are able to select the desired offering and service commitment levels via the ServiceNow® Service Catalog. |
| Technical service | A service type that is published to service owners and typically underpins a business or application service. | Typically orderable by service owners. Service owners are able to select the desired offering and service commitment levels via the Service Catalog. |
| Application service | A service type that is a logical representation of a deployed application stack. | Examples of application services are hosting, data backup, and recovery. |
| Application | Any deployed program, module, or group of programs that is designed to provide specific functionality on a computer infrastructure. | Defines behavior and has specific functionality associated with it. Applications are typically discoverable functionality, like Apache Web Server. |
| Business Application | Represents all software and infrastructure environments (dev, test, prod) configured to provide functionality. | Used to increase productivity and perform other business functions accurately. |
| Service catalog | Provides consumable view of available products, services, service commitment options, and offerings. | Helps manage what services a user may have access to. Also, catalogs are the initiation point for access to available services. |
| Service offering | A stratification of a service into capability, availability, pricing, and packaging options. | Different levels of performance and features for a given service can be made available. |
| Service commitment | Defines service delivery obligations agreed to between the consumer and the provider. | Often manifested in the form of contracts such as service level agreements, operational level agreements, and underpinning contracts. |
| Configuration item (CI) | Physical and logical components of an infrastructure that are currently or soon will be under configuration management. | May be a single module such as a server, database, or router or a more complex item, such as a complete system. |
| Asset | An item whose financial value is tracked. | Many assets are CIs and vice versa, but that is not always the case. Assets have a life cycle with financial considerations. |
| Operating model | An abstract and ideally visual representation (model) of how an organization delivers value to its customers or beneficiaries. | Typically represents the various elements of how an organization operates. It usually incorporates strategy positions such as the innovation model, degree of intelligent automation, industry alignment, provider delivery models, and the business expectations of IT. |
"""

In [ ]:
# markdown_table = """
# | Metric Group       | Metrics                       | Description                                                                                      |
# |--------------------|-------------------------------|--------------------------------------------------------------------------------------------------|
# | Base Metrics       | Attack Vector (AV)            | Describes how an attacker can access the vulnerable component.                                   |
# |                    | Attack Complexity (AC)        | Represents the level of complexity required to exploit the vulnerability.                        |
# |                    | Privileges Required (PR)      | Indicates the privileges an attacker needs to exploit the vulnerability.                         |
# |                    | User Interaction (UI)         | Describes whether user interaction is required for the vulnerability to be exploited.            |
# |                    | Scope (S)                     | Specifies whether the vulnerability impacts only the vulnerable component or also affects others.|
# |                    | Confidentiality (C) Impact    | Represents the impact on confidentiality if the vulnerability is successfully exploited.         |
# |                    | Integrity (I) Impact          | Represents the impact on integrity if the vulnerability is successfully exploited.               |
# |                    | Availability (A) Impact       | Represents the impact on availability if the vulnerability is successfully exploited.            |
# |                    | Exploit Code Maturity (E)     | Reflects the current state of exploit techniques and the availability of exploit code.           |
# | Temporal Metrics   | Exploitability (Exploitability) | Represents the likelihood of the vulnerability being exploited.                                |
# |                    | Remediation Level (RL)        | Reflects the availability and effectiveness of existing remedies or workarounds.                 |
# |                    | Report Confidence (RC)        | Indicates the level of confidence in the existence of the vulnerability and its impact.          |
# | Environmental Metrics | Confidentiality (CR) Requirements | Represents the importance of confidentiality to the impacted organization.                |
# |                    | Integrity (IR) Requirements   | Represents the importance of integrity to the impacted organization.                             |
# |                    | Availability (AR) Requirements| Represents the importance of availability to the impacted organization.                          |
# |                    | Modified Impact (MI)          | Reflects the potential impact on confidentiality, integrity, and availability, considering mitigating factors.|
# """

In [3]:
# Split markdown table into lines
markdown_lines = markdown_table.split("\n")[3:-1]

# Create a pandas DataFrame
dataframe = pd.read_csv(StringIO('\n'.join(markdown_lines)), sep='|', header=None)

# Remove white spaces
dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Set column names manually
dataframe.columns = ['Unnamed: 0', 'Table Name (UI)', 'Technical Name', 'Description', 'Unnamed: 4']

# Drop empty columns
dataframe = dataframe.dropna(axis=1)

# Create a workbook and add a worksheet to it
wb = Workbook()
ws = wb.active

# Define fills for cells
header_fill = PatternFill(start_color="182546",
                           end_color="182546",
                           fill_type="solid")

# Define fonts for cells
header_font = Font(color="FFFFFF", bold=True)
default_font = Font(color="000000")

# Define cell alignment
alignment = Alignment(horizontal='left', vertical='center', wrap_text=True)

# Add data to worksheet
for i, column in enumerate(dataframe.columns):
    cell = ws.cell(row=1, column=i+1)
    cell.value = column
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = alignment

for r, row in enumerate(dataframe_to_rows(dataframe, index=False, header=False), start=2):
    for c, value in enumerate(row, start=1):
        cell = ws.cell(row=r, column=c, value=value)
        cell.font = default_font
        cell.alignment = alignment

# Autofit column widths
for column_cells in ws.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    column_cells[0].parent.column_dimensions[column_cells[0].column_letter].width = length

# Save the workbook
wb.save("markdown_converted_output.xlsx")


Less efficient method
* Convert to dataframe
* Convert to html (then open in browser)

In [4]:
# import pandas as pd
# from io import StringIO
# from IPython.display import display, HTML

# markdown_table = """
# | Table Name (UI) | Technical Name | Description |
# | --- | --- | --- |
# | Business Unit | cmn_cost_center | Contains records for each business unit within your organization. Business units can be used for categorizing costs, work, and other elements at a more granular level than a company. |
# | Department | cmn_department | Contains records for each department within your organization. Each user can be assigned to a department, which can be used for routing decisions, reporting, and other functions. |
# | Location | cmn_location | Contains records for physical locations where your company operates. This can include offices, data centers, or other locations. Each user and asset can be associated with a location. |
# | Company | core_company | Contains records for each company that your organization interacts with. This can include your own company, vendors, partners, and customers. |
# | User | sys_user | Contains a record for each individual who has (or had) access to your ServiceNow instance. It includes information like their name, email, roles, etc. |
# | Group | sys_user_group | Contains records for each group of users within your ServiceNow instance. Groups can be used for many purposes, like assignment, approval rules, notifications, and access control. |
# | Group Members | sys_user_grmember | Contains records of the relationships between users and groups. Each record represents a user being a member of a group. |
# | Role | sys_user_role | Contains records for each role within your ServiceNow instance. Roles are used to define what actions a user can perform and what data they can access. |
# """

# # Split markdown table into lines
# markdown_lines = markdown_table.split("\n")[3:-1]

# # Create a pandas DataFrame
# dataframe = pd.read_csv(StringIO('\n'.join(markdown_lines)), sep='|', header=None)

# # Remove white spaces
# dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# # Set column names manually
# dataframe.columns = ['Unnamed: 0', 'Table Name (UI)', 'Technical Name', 'Description', 'Unnamed: 4']

# # Drop empty columns
# dataframe = dataframe.dropna(axis=1)

# dataframe

# # # Convert the DataFrame to HTML and display
# # html_table = dataframe.to_html(index=False)
# # display(HTML(html_table))


In [5]:
# # Save the DataFrame to HTML and save as a file
# html_table = dataframe.to_html(index=False)
# with open('itsm_foundation_data_table.html', 'w') as f:
#     f.write(html_table)
